In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys

#### 단일 cell 을 통한 계산 처리 과정 : 기본 사용법

In [11]:
# 입력 데이터 준비
inputs = np.array([[[1, 2]]])

In [12]:
import warnings
warnings.filterwarnings(action="ignore")

tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype=tf.float32)

# 1 * 3의 matrix 출력([6, 7, 8] 과 같은)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, 
                                   inputs=tf_inputs)

print(outputs)
print(state)

print("\n Weights")
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(w)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)

 Weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    
    print("Output Value")
    print(output_run)
    
    print("State Value")
    print(state_run)
    
    variables_names = [v.name for v in tf.trainable_variables()]
    print(variables_names)
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print(k, v)
    

Output Value
[[[-0.9314169   0.75578666 -0.6819246 ]]]
State Value
[[-0.9314169   0.75578666 -0.6819246 ]]
['rnn/basic_rnn_cell/kernel:0', 'rnn/basic_rnn_cell/bias:0']
rnn/basic_rnn_cell/kernel:0 [[-0.62831575  0.38538355  0.79733914]
 [-0.5203329   0.30046564 -0.8150209 ]
 [ 0.39399797  0.16670114  0.4062907 ]
 [-0.6391754   0.8460203   0.5266966 ]
 [ 0.41124135  0.66347724 -0.0210759 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


#### 다중 cell

In [14]:
# I work at google = [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]]
# I google at work = [[1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0]]

inputs = np.array([
                [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]],
                [[1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0]]
                ])

In [15]:
tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype=tf.float32)

rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, 
                                  inputs=tf_inputs)
variables_names = [v.name for v in tf.trainable_variables()]

print(outputs)
print("--------------------------------------------------------------")
print(state)
print("--------------------------------------------------------------")
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(w)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
--------------------------------------------------------------
Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
--------------------------------------------------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    
    print("Output Value")
    print(output_run)
    
    print("State Value")
    print(state_run)
    
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print(k, v)

Output Value
[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324395]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641277  0.70696247 -0.7512605 ]]]
State Value
[[-0.36491966  0.8857268  -0.02324395]
 [-0.11641277  0.70696247 -0.7512605 ]]
rnn/basic_rnn_cell/kernel:0 [[-0.56198275  0.34469748  0.7131618 ]
 [-0.4653999   0.2687447  -0.7289769 ]
 [ 0.35240245  0.14910203  0.36339748]
 [-0.57169586  0.7567036   0.47109187]
 [ 0.3678255   0.5934322  -0.01885086]
 [ 0.31208777 -0.40880746  0.22867584]
 [ 0.5521256   0.682691   -0.5481483 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


In [17]:
tf.reset_default_graph()
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=3)

x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, 
                                  inputs=x_data)

variables_names = [v.name for v in tf.trainable_variables()]

print(outputs)
print("--------------------------------------------------------------")
print(state)
print("--------------------------------------------------------------")
for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(w)

Tensor("rnn/transpose_1:0", shape=(1, 5, 3), dtype=float32)
--------------------------------------------------------------
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
--------------------------------------------------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    
    print("Output Value")
    print(output_run)
    
    print("State Value")
    print(state_run)
    
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print(k, v)

Output Value
[[[ 0.03521261 -0.55469215  0.3911671 ]
  [ 0.71793234 -0.6416993  -0.23057865]
  [ 0.28477573 -0.04065027 -0.6732457 ]
  [ 0.27927202 -0.02790875 -0.29825467]
  [ 0.5107646   0.44046617 -0.58083504]]]
State Value
[[ 0.5107646   0.44046617 -0.58083504]]
rnn/basic_rnn_cell/kernel:0 [[ 0.03522718 -0.62513345  0.41317725]
 [ 0.7187083  -0.7458999   0.24596906]
 [ 0.07847679 -0.2686134  -0.3954072 ]
 [ 0.5364299   0.36027014 -0.70091194]
 [-0.4692109   0.02520639  0.03773957]
 [-0.691509   -0.20686752  0.75621474]
 [-0.4662865  -0.3343653  -0.16011602]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


#### hihello 학습 과정

In [19]:
idx2char = ['h','i','e','l','o']
x_one_hot = [[[1,0,0,0,0],
             [0,1,0,0,0],
             [1,0,0,0,0],
             [0,0,1,0,0],
             [0,0,0,1,0],
             [0,0,0,1,0],
             [0,0,0,0,1]]]

# x_data = [[0,1,0,2,3,3]] #hihell
y_data = [[0,1,0,2,3,3,4]] #hihello

In [20]:
# parameter
sequence_len = 7 # x_one_hot의 행개수
num_classes = 5
batch_size = 1

In [21]:
tf.reset_default_graph()

# X, y 준비
X = tf.placeholder(tf.float32, shape=[None, sequence_len, num_classes])
y = tf.placeholder(tf.int32, shape=[None, sequence_len])

In [22]:
# rnn 모델
cell = tf.contrib.rnn.BasicRNNCell(num_units=num_classes)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X, dtype=tf.float32)
print(outputs)
print(state)

Tensor("rnn/transpose_1:0", shape=(?, 7, 5), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(?, 5), dtype=float32)


In [23]:
# 평면화(Flat Layer)
X_for_fc = tf.reshape(outputs, [-1, num_classes])
print(X_for_fc)

Tensor("Reshape:0", shape=(?, 5), dtype=float32)


In [24]:
# FC (W, b, logit)
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, 
                                  activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_len, num_classes])
W = tf.ones([batch_size, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, 
                                        targets=y, weights=W))
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        _, c = sess.run([train, cost], feed_dict={X:x_one_hot, y:y_data})
        result = sess.run(tf.argmax(outputs, 2), feed_dict={X:x_one_hot})
        print(i, 'cost:', c, "   prediction:", result,"    정답:",y_data)
        
    result_str = [idx2char[c] for c in np.squeeze(result)]
    print('결과 :',', '.join(result_str))

Instructions for updating:
Please use `layer.__call__` method instead.
0 cost: 1.5103203    prediction: [[0 1 0 3 3 3 3]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
1 cost: 1.1951942    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
2 cost: 0.92564857    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
3 cost: 0.6775916    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
4 cost: 0.49333224    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
5 cost: 0.35018376    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
6 cost: 0.25466552    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
7 cost: 0.18750243    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
8 cost: 0.13522802    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
9 cost: 0.09569024    prediction: [[0 1 0 2 3 3 4]]     정답: [[0, 1, 0, 2, 3, 3, 4]]
결과 : h, i, h, e, l, l, o


#### 두 번째 예제

In [25]:
#sample = " You will have to submit to your fate whether you will or not."
sample = " if you want you"
idx2char = list(set(sample))
print(idx2char)

char2idx = {c: i for i, c in enumerate(idx2char)}
print(char2idx)

sample_idx = [char2idx[c] for c in sample]
print(sample_idx)

x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

['u', 'n', 'i', 'a', 'f', ' ', 'w', 't', 'o', 'y']
{'u': 0, 'n': 1, 'i': 2, 'a': 3, 'f': 4, ' ': 5, 'w': 6, 't': 7, 'o': 8, 'y': 9}
[5, 2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]


In [26]:
# parameter
num_classes = len(char2idx)
batch_size = 1
sequence_len = len(sample) -1

In [27]:
tf.reset_default_graph()

# X, y 준비
X = tf.placeholder(tf.int32, shape=[None, sequence_len])
y = tf.placeholder(tf.int32, shape=[None, sequence_len])

x_one_hot = tf.one_hot(X, num_classes)

In [28]:
cell = tf.contrib.rnn.BasicRNNCell(num_units=num_classes)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=x_one_hot, dtype=tf.float32)

W = tf.ones([batch_size, sequence_len])
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=outputs, 
                                                        targets=y, weights=W))
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        _, c = sess.run([train, cost], feed_dict={X:x_data, y:y_data})
        result = sess.run(tf.argmax(outputs, 2), feed_dict={X:x_data})
        print(i, 'cost:', c, "   prediction:", result,"    정답:",y_data)
        
    result_str = [idx2char[c] for c in np.squeeze(result)]
    print('결과 :',', '.join(result_str))

0 cost: 2.3314052    prediction: [[6 2 2 9 5 2 5 0 9 2 6 6 9 5 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
1 cost: 2.0096936    prediction: [[9 8 5 9 5 0 5 0 9 5 1 6 9 5 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
2 cost: 1.8021482    prediction: [[9 8 5 9 5 0 5 6 9 5 7 5 9 5 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
3 cost: 1.6359936    prediction: [[9 8 5 9 8 0 5 6 9 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
4 cost: 1.5283053    prediction: [[9 8 5 9 8 0 5 6 8 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
5 cost: 1.4471759    prediction: [[9 8 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
6 cost: 1.3742923    prediction: [[9 8 5 9 8 0 5 6 8 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
7 cost: 1.3016006    prediction: [[9 8 5 9 8 0 5 6 8 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
8 cost: 1.255878

69 cost: 0.81537783    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
70 cost: 0.8151085    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
71 cost: 0.8148612    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
72 cost: 0.81462485    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
73 cost: 0.8143907    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
74 cost: 0.814175    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
75 cost: 0.81396645    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
76 cost: 0.8137617    prediction: [[2 4 5 9 8 0 5 6 3 1 7 5 9 8 0]]     정답: [[2, 4, 5, 9, 8, 0, 5, 6, 3, 1, 7, 5, 9, 8, 0]]
77 cos

## LSTM(Long Short Term Memory)

+ BackProgation Through TIme(BPTT)
    - Gradient Vanishing
    - Gradient Exploding

In [9]:
tf.reset_default_graph()
tf.set_random_seed(777)

inputs = np.array([[[1,0]]])

tf_inputs = tf.constant(inputs, dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units=1)
outputs, states = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs = tf_inputs)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run, states_run = sess.run([outputs, states])
    
    print(outputs_run)
    print(states_run.h)
    print(states_run.c)

[[[0.09927537]]]
[[0.09927537]]
[[0.18134572]]


## Dynamic RNN

In [10]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

df = pd.DataFrame(paragraph_dict_list)
df = df[["paragraph", "category"]]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


#### Preprocessing

In [31]:
results = set()
df["paragraph"].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [32]:
results

{'(and',
 '1',
 '1.5',
 '1.5x',
 '10-9',
 '14',
 '2',
 '2014',
 '33/1',
 'a',
 'about',
 'action',
 'after',
 'against',
 'ago',
 'ai',
 'alisson',
 'all',
 'almost',
 'also',
 'although',
 'alto',
 'amazing',
 'american',
 'and',
 'angelique',
 'any',
 'apps',
 'are',
 'area',
 'around',
 'as',
 'at',
 'australia',
 'avoided',
 'azarenka',
 'back',
 'battle',
 'be',
 'beef',
 'been',
 'before',
 'began',
 'began—too',
 'being',
 'better',
 'between',
 'big',
 'biscuits',
 'bit',
 'bookmark',
 'bookmarked',
 'both',
 'bottom',
 'brazilian',
 'bread',
 'breakthroughs',
 'briton',
 'brunch',
 'burger',
 'burgers',
 'business',
 'busy',
 'but',
 'by',
 'came',
 'campaigns',
 'can',
 'card',
 'caroline',
 'case',
 'certainly',
 'champion',
 'champions',
 'chance',
 'chances',
 'chicken',
 'city',
 'clay',
 'clean',
 'commented',
 'competition',
 'complain',
 'confidence',
 'contrast',
 'could',
 'course',
 'courteous',
 'courts',
 'coworkers',
 'crisp',
 'croatia',
 'crumbs',
 'culminating

In [34]:
idx2word=dict(enumerate(results))
print(idx2word[396])
print(idx2word[100])

results
alto


In [35]:
word2idx={v:k for k, v in idx2word.items()}
print(word2idx["results"])
print(word2idx["alto"])

396
100


In [43]:
def encoded_paragraph(para):
    words = para.split()
    encoded=[]
    for w in words:
        encoded.append([word2idx[w]])
        
    return encoded

def encoded_category(cate):
    if cate=="food":
        return [1,0]
    else:
        return [0,1]
    
def 

In [44]:
test = df.paragraph.apply(encoded_paragraph)
test

0     [[367], [152], [293], [276], [339], [165], [92...
1     [[196], [262], [201], [428], [527], [229], [26...
2     [[119], [283], [491], [191], [493], [392], [90...
3     [[518], [497], [321], [121], [480], [274], [35...
4     [[274], [125], [90], [269], [192], [360], [150...
5     [[274], [143], [348], [2], [40], [304], [460],...
6     [[232], [446], [192], [82], [441], [475], [274...
7     [[219], [441], [13], [157], [473], [470], [100...
8     [[274], [8], [15], [139], [130], [17], [40], [...
9     [[68], [274], [317], [449], [5], [102], [274],...
10    [[274], [112], [79], [50], [474], [172], [294]...
11    [[274], [14], [192], [492], [291], [27], [251]...
12    [[412], [259], [369], [489], [512], [421], [41...
13    [[46], [231], [15], [387], [381], [189], [276]...
14    [[274], [62], [242], [526], [420], [163], [397...
15    [[274], [509], [331], [486], [276], [136], [10...
16    [[256], [528], [213], [178], [387], [105], [89...
17    [[206], [361], [500], [256], [528], [421],

In [45]:
df.category.apply(encoded_category)

0     [1, 0]
1     [1, 0]
2     [1, 0]
3     [1, 0]
4     [1, 0]
5     [1, 0]
6     [1, 0]
7     [1, 0]
8     [1, 0]
9     [1, 0]
10    [0, 1]
11    [0, 1]
12    [0, 1]
13    [0, 1]
14    [0, 1]
15    [0, 1]
16    [0, 1]
17    [0, 1]
18    [0, 1]
19    [0, 1]
Name: category, dtype: object

In [ ]:
df["seq_length"] = df.paragraph.apply(word_cnt)
df.head()

In [46]:
# 문장의 최대 길이 알아내기
max_word_cnt = 0

for row in df["paragraph"]:
    if len(row.split())> max_word_cnt:
        max_word_cnt = len(row.split())
        
print(max_word_cnt)        

91


In [52]:
# 문장의 길이가 91개가 안될 경우 -1로 채워준다
def sequence_padding(enc_para):
    seq_len = len(enc_para)
    for i in range(seq_len, max_word_cnt):
        enc_para.append([-1])
        
    return enc_para


In [53]:
df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

AttributeError: 'DataFrame' object has no attribute 'enc_paragraph'

In [49]:
print(df["enc_paragraph"][0])

KeyError: 'enc_paragraph'

In [54]:
# 입력값들을 배열로 변환
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.cnc.category.tolist())
seq_length = np.array(df.seq_length.tolist())

AttributeError: 'DataFrame' object has no attribute 'enc_paragraph'

### 모델 구축

In [56]:
train_X = enc_paragraph
train_y = enc_category

print(train_X.shape, train_y.shape)

NameError: name 'enc_paragraph' is not defined

In [59]:
tf.reset_default_graph()
tf.set_random_seed(777)

#parameter
lr = 0.001
n_epochs = 300

X = tf.placeholder(tf.float32, [None, max_word_cnt,1])
y = tf.placeholder(tf.int32, [None, 2])

embeded = tf.layer.dense(X,5)

cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
output, state = tf.nn.dynamic_rnn(cell=cell, input=X, dtype=tf.float32,
                                 sequence_length=seq_length)

#print(output)
#print(state)

first_layer = tf.layers.dense(state,h,32)
logit = tf.layers.dense(first_layer, 2)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, lab))
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

#print(X)
#print(state)
#print(first_layer)
#print(logit)
#print(embeded)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        _,c = sess.run([train,cost], feed_dict={X:train_X, y:train_y})
        
        if epoch % 50 == 0:
            pred=tf.nn.softmax(logit)
            correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
            accuracy=tf.reduce_mean(tf.cast(correct_pred, "float"))
            cur_acc = accuracy.eval({X:train_X, y:train_y})
            print("epoch:"+str(epoch)+ ", cost:" +str(cost)+ ", acc:" +str(cur_acc))

NameError: name 'seq_length' is not defined

In [ ]:
# https://insightcampus.co.kr/insightcommunity/?mod=document&uid=12936
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

tf.reset_default_graph()

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = 'log_dir/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = 'data/belling_the_cat.txt'

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    
    return content

training_data = read_data(training_file)
print("Loaded training data...")

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)

    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")